Применимость методов BFGS/L-BFGS:

Эти алгоритмы гарантированно сходятся для дважды дифференцируемых выпуклых функций. Проверим нашу функцию на соответствие этим требованиям.

In [ ]:
import numpy as np
from utils import bfgs, lbfgs

def target_function(x):
    """
    Целевая функция для минимизации:
    f(x) = 0.5 * [x₁² + (x₁-x₂)² + (x₂-x₃)² + x₃²] - x₁
    """
    x1, x2, x3 = x[0], x[1], x[2]
    # Более простая для вычислений форма: f(x) = x₁² + x₂² + x₃² - x₁x₂ - x₂x₃ - x₁
    return x1**2 + x2**2 + x3**2 - x1*x2 - x2*x3 - x1

def analytical_gradient(x):
    """
    Аналитически вычисленный градиент целевой функции.
    """
    x1, x2, x3 = x[0], x[1], x[2]
    df_dx1 = 2 * x1 - x2 - 1
    df_dx2 = 2 * x2 - x1 - x3
    df_dx3 = 2 * x3 - x2
    return np.array([df_dx1, df_dx2, df_dx3])

def numerical_gradient(func, x, h=1e-7):
    """
    Численное вычисление градиента с использованием центральной разности (формула 7).
    """
    grad = np.zeros_like(x)
    for i in range(len(x)):
        # Создаем векторы x+h*eᵢ и x-h*eᵢ
        x_plus_h = x.copy()
        x_plus_h[i] += h
        x_minus_h = x.copy()
        x_minus_h[i] -= h
        
        # Применяем формулу центральной разности
        grad[i] = (func(x_plus_h) - func(x_minus_h)) / (2 * h)
    return grad
    

In [9]:
# Начальная точка для всех методов
x0 = np.array([0.0, 0.0, 0.0])
print("="*50)
print("Запуск BFGS с аналитическим градиентом")
print("="*50)
x_final_bfgs_analytic, f_min_bfgs_analytic, _ = bfgs(target_function, analytical_gradient, x0)
print(f"Найденный минимум x: {x_final_bfgs_analytic}")
print(f"Значение функции в минимуме f(x): {f_min_bfgs_analytic}\n")

Запуск BFGS с аналитическим градиентом
BFGS сошелся за 7 итераций.
Найденный минимум x: [0.75       0.5        0.24999999]
Значение функции в минимуме f(x): -0.3749999999999999



In [10]:
print("Запуск BFGS с численным градиентом")
# Оборачиваем grad_f в lambda, чтобы передать только один аргумент x
x_final_bfgs_numeric, f_min_bfgs_numeric, _ = bfgs(target_function, lambda x: numerical_gradient(target_function, x), x0)
print(f"Найденный минимум x: {x_final_bfgs_numeric}")
print(f"Значение функции в минимуме f(x): {f_min_bfgs_numeric}\n")

Запуск BFGS с численным градиентом
BFGS сошелся за 7 итераций.
Найденный минимум x: [0.75       0.5        0.24999999]
Значение функции в минимуме f(x): -0.375



In [11]:
print("Запуск L-BFGS с аналитическим градиентом (m=5)")
x_final_lbfgs_analytic, f_min_lbfgs_analytic, _ = lbfgs(target_function, analytical_gradient, x0, m=5)
print(f"Найденный минимум x: {x_final_lbfgs_analytic}")
print(f"Значение функции в минимуме f(x): {f_min_lbfgs_analytic}\n")

Запуск L-BFGS с аналитическим градиентом (m=5)
L-BFGS (m=5) сошелся за 7 итераций.
Найденный минимум x: [0.75       0.5        0.24999999]
Значение функции в минимуме f(x): -0.37499999999999994



In [12]:
print("Запуск L-BFGS с численным градиентом (m=5)")
x_final_lbfgs_numeric, f_min_lbfgs_numeric, _ = lbfgs(target_function, lambda x: numerical_gradient(target_function, x), x0, m=5)
print(f"Найденный минимум x: {x_final_lbfgs_numeric}")
print(f"Значение функции в минимуме f(x): {f_min_lbfgs_numeric}\n")

Запуск L-BFGS с численным градиентом (m=5)
L-BFGS (m=5) сошелся за 7 итераций.
Найденный минимум x: [0.75       0.5        0.24999999]
Значение функции в минимуме f(x): -0.375



In [13]:
# Аналитическое решение для сравнения
# H * x = b, где b = [1, 0, 0]^T
# [[2, -1, 0], [-1, 2, -1], [0, -1, 2]] * [x1, x2, x3]^T = [1, 0, 0]^T
# Решение системы: x1=0.75, x2=0.5, x3=0.25
H_inv = np.linalg.inv(np.array([[2, -1, 0], [-1, 2, -1], [0, -1, 2]]))
b = np.array([1, 0, 0])
analytical_solution = H_inv.dot(b)
print("="*50)
print(f"Аналитическое решение: {analytical_solution}")
print(f"Значение функции в точке аналитического решения: {target_function(analytical_solution)}")
print("="*50)

Аналитическое решение: [0.75 0.5  0.25]
Значение функции в точке аналитического решения: -0.375
